1. KHAI BÁO THƯ VIỆN

In [6]:
import pandas as pd
import re
from textblob import TextBlob

2. LOAD DATA VÀ XEM DỮ LIỆU

In [7]:
data = pd.read_csv("D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\data_source\sentimentdataset.csv")
df = pd.DataFrame(data)
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HaoPham\AppData\Local\Temp\ipykernel_10468\3030248052.py:1: SyntaxWarning: invalid escape sequence '\D'
  data = pd.read_csv("D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\data_source\sentimentdataset.csv")


,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


3. KIỂM TRA VÀ TÌM HIỂU DỮ LIỆU 

In [8]:
# Thông tin về dữ liệu
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  732 non-null    int64  
 1   Unnamed: 0    732 non-null    int64  
 2   Text          732 non-null    object 
 3   Sentiment     732 non-null    object 
 4   Timestamp     732 non-null    object 
 5   User          732 non-null    object 
 6   Platform      732 non-null    object 
 7   Hashtags      732 non-null    object 
 8   Retweets      732 non-null    float64
 9   Likes         732 non-null    float64
 10  Country       732 non-null    object 
 11  Year          732 non-null    int64  
 12  Month         732 non-null    int64  
 13  Day           732 non-null    int64  
 14  Hour          732 non-null    int64  
dtypes: float64(2), int64(6), object(7)
memory usage: 85.9+ KB


In [9]:
# Thống kê cơ bản
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0.1,732.0,366.464481,211.513936,0.0,183.75,366.5,549.25,732.0
Unnamed: 0,732.0,369.740437,212.428936,0.0,185.75,370.5,553.25,736.0
Retweets,732.0,21.508197,7.061286,5.0,17.75,22.0,25.00,40.0
Likes,732.0,42.901639,14.089848,10.0,34.75,43.0,50.00,80.0
Year,732.0,2020.471311,2.802285,2010.0,2019.00,2021.0,2023.00,2023.0
Month,732.0,6.122951,3.411763,1.0,3.00,6.0,9.00,12.0
Day,732.0,15.497268,8.474553,1.0,9.00,15.0,22.00,31.0
Hour,732.0,15.521858,4.113414,0.0,13.00,16.0,19.00,23.0


In [10]:
# Kiểm tra dữ liệu null
df.isnull().sum()

Unnamed: 0.1    0
Unnamed: 0      0
Text            0
Sentiment       0
Timestamp       0
User            0
Platform        0
Hashtags        0
Retweets        0
Likes           0
Country         0
Year            0
Month           0
Day             0
Hour            0
dtype: int64

In [11]:
# Kiểm tra dữ liệu trùng lặp
df.duplicated().sum()

np.int64(0)

In [12]:
# Tổng quan 
df.shape

(732, 15)

Sau khi kiểm tra dữ liệu thì có nhưng điểm chính như sau:

    + Tổng quang: có 732 dòng và 15 cột
    + Gía trị thiếu: không có giá trị thiếu
    + Các trường dữ liệu: trong đó chỉ có 1 trường là phân loại: 'Timestamp'(theo dạng ngày),
    còn lại hậu như là theo dạng số và chuỗi.
    + Ngoại lệ: không có các ngoại lệ
    + Các dữ liệu trùng lập: không có dữ liệu trùng lập
-----------------------------------------------------------------
nhận xét: dữ liệu hiện đang có 2 cột không xác định cần loại bỏ. Cũng như dữ liệu trong mỗi dòng ở cột 'Text' có chữa các biểu tượng cảm xúc, những cái này sẽ gây ra lỗi khi up nó lên SQL. Sau cùng là định dạng dữ liệu ở cột 'Timestamp' lại thành Datetime.

4. Xử lí dữ liệu

4.1 Xóa các dữ liệu lỗi

In [13]:
# Xóa cột 'Unnamed: 0.1', 'Unnamed: 0'
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [14]:
# Xóa các kí tự đặc biệt
# Hàm loại bỏ ký tự đặc biệt, emoji và giữ lại chữ cái, chữ số
def clean_text(text):
    # Sử dụng regex để loại bỏ ký tự đặc biệt và emoji
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Chỉ giữ lại chữ cái và chữ số
    text = re.sub(r'\s+', ' ', text).strip()  # Xóa khoảng trắng thừa
    return text

# Áp dụng hàm vào cột 'Text'
df['Text'] = df['Text'].apply(clean_text)

In [15]:
# Xóa khoảng trắng ở đầu và cuối các chuỗi trong các trường dữ liệu text
# Xóa khoảng trắng ở đầu và cuối các chuỗi trong cột 'Platform'
df['Platform'] = df['Platform'].str.strip()
# Xóa khoảng trắng ở đầu và cuối các chuỗi trong cột 'Country'
df['Country'] = df['Country'].str.strip()
# Xóa khoảng trắng ở đầu và cuối các chuỗi trong cột 'Sentiment'
df['Sentiment'] = df['Sentiment'].str.strip()
# Xóa khoảng trắng ở đầu và cuối các chuỗi trong cột 'Hashtags'
df['Hashtags'] = df['Hashtags'].str.strip()
# Xóa khoảng trắng ở đầu và cuối các chuỗi trong cột 'User'
df['User'] = df['User'].str.strip()
# Xóa khoảng trắng ở đầu và cuối các chuỗi trong cột 'Text'
df['Text'] = df['Text'].str.strip()

4.2. Chuẩn hóa dữ liệu

In [16]:
# Đổi định dạng cột Timestamp thành kiểu datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
# Đổi định dạng cột Likes, Retweets thành kiểu int64
df[['Likes', 'Retweets']] = df[['Likes', 'Retweets']].astype('int64')

In [17]:
# Giới hạn chiều dài chuỗi tối đa 255 ký tự của cột Text
df['Text'] = df['Text'].str.slice(0, 255)

4.3 Data sau khi xử lí

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       732 non-null    object        
 1   Sentiment  732 non-null    object        
 2   Timestamp  732 non-null    datetime64[ns]
 3   User       732 non-null    object        
 4   Platform   732 non-null    object        
 5   Hashtags   732 non-null    object        
 6   Retweets   732 non-null    int64         
 7   Likes      732 non-null    int64         
 8   Country    732 non-null    object        
 9   Year       732 non-null    int64         
 10  Month      732 non-null    int64         
 11  Day        732 non-null    int64         
 12  Hour       732 non-null    int64         
dtypes: datetime64[ns](1), int64(6), object(6)
memory usage: 74.5+ KB


In [19]:
df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,Traffic was terrible this morning,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,Just finished an amazing workout,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


4.4 Tạo df1 từ df đã có

In [20]:
df1 = df.copy()

4.5 Phân loại các cảm xúc thành 3 nhóm cảm xúc chính

In [21]:
# Hàm phân loại cảm xúc
def classify_sentiment(text):
    # Giữ nguyên nếu là Neutral
    if text.lower() == 'neutral':
        return 'Neutral'
    else:
        # Sử dụng TextBlob để xác định polarity cho các giá trị khác
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity
        
        # Phân loại dựa trên polarity
        if polarity > 0:
            return 'Positive'
        else:
            return 'Negative'

# Áp dụng hàm phân loại cho cột Sentiment
df1['Summarize Sentiment'] = df1['Sentiment'].apply(classify_sentiment)


In [22]:
# Xóa cột Sentiment
df1 = df1.drop(['Sentiment'], axis=1)

In [23]:
# # Đổi vị trí cột Summarize Sentiment
Summarize_Sentiment = df1.pop('Summarize Sentiment')
df1.insert(1, 'Summarize Sentiment', Summarize_Sentiment)

---
---

In [24]:
# Tách các hashtag bằng dấu phẩy hoặc khoảng trắng
# df1['Hashtags_Split'] = df1['Hashtags'].str.split('[, ]+')  # Sử dụng regex để tách bằng dấu phẩy hoặc khoảng trắng
# # Chuyển tất cả hashtag thành một danh sách duy nhất
# hashtags = df1['Hashtags_Split'].dropna().sum()  # Gộp các danh sách con thành một danh sách lớn
# hashtags = pd.DataFrame(hashtags)

In [25]:
df1.head()

,Text,Summarize Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,Traffic was terrible this morning,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,Just finished an amazing workout,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


In [26]:
df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,Traffic was terrible this morning,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,Just finished an amazing workout,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


In [27]:
df1.shape

(732, 13)

Tổng quan các bước xử lý dữ liệu:

1. Xóa Các dữ liệu lỗi:

1.1. Xóa cột 'Unnamed: 0.1', 'Unnamed: 0'

1.2 Xóa các ký tự đặc biệt ở cột "Text"

1.3 Xóa khoảng trắng ở đầu và cuối các chuỗi trong các trường dữ liệu có kiểu "text"

2. Chuẩn hóa dữ liệu:

2.1 Đổi định dạng cột 'Timestamp' thành kiểu datetime

2.2 Đổi định dạng cột 'Likes', 'Retweets' thành kiểu int64

2.3 Giới hạn chiều dài chuỗi tối đa 255 ký tự của cột Text

3. Tạo dataframe mới (df1):

3.1 Phân loại các cảm xúc ở cột 'Sentiment' thành 3 nhóm cảm xúc chính:

 3.1.1 Tạo cột 'Sumerize Sentiment' chứa values là dữ liệu đã phân loại ở cột 'Sentiment'

3.2 Xóa cột 'Sentiment'

3.3 Đổi vị trí cột 'Sumerize Sentiment'


5. Lưu dữ liệu về file csv

In [28]:
# Lưu df đã xử lí
df.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset.csv", index=False)
df1.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset01.csv", index=False)
# hashtags.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\hashtags.csv", index=False, header=False)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HaoPham\AppData\Local\Temp\ipykernel_10468\3121500483.py:2: SyntaxWarning: invalid escape sequence '\D'
  df.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset.csv", index=False)
C:\Users\HaoPham\AppData\Local\Temp\ipykernel_10468\3121500483.py:3: SyntaxWarning: invalid escape sequence '\D'
  df1.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset01.csv", index=False)
C:\Users\HaoPham\AppData\Local\Temp\ipykernel_10468\3121500483.py:2: SyntaxWarning: invalid escape sequence '\D'
  df.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset.csv", index=False)
C:\Users\HaoPham\AppData\Local\Temp\ipykernel_10468\3121500483.py:3: SyntaxWarning: invalid escape sequence '\D'
  df1.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sent

OSError: Cannot save file into a non-existent directory: 'E:\DATN\Sentiment_Analysis\data\cleaned_data'

In [391]:
# Đọc dữ liệu từ file CSV
data_json = pd.read_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset01.csv")

# Chuyển đổi DataFrame sang định dạng JSON
data_json.to_json("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset.json", orient='records', lines=True)

print("Đã chuyển đổi CSV thành JSON thành công!")

Đã chuyển đổi CSV thành JSON thành công!


<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\KHANH\AppData\Local\Temp\ipykernel_50568\2703791328.py:2: SyntaxWarning: invalid escape sequence '\D'
  data_json = pd.read_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset01.csv")
C:\Users\KHANH\AppData\Local\Temp\ipykernel_50568\2703791328.py:5: SyntaxWarning: invalid escape sequence '\D'
  data_json.to_json("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset.json", orient='records', lines=True)
